In [1]:
from lp_random_gen import random_lp_int
from docplex.mp.advmodel import AdvModel
from docplex.mp.model_reader import ModelReader
from helpers import *
getPath = lambda fn : 'SYNTH_DATA/'+fn+".lp"

In [2]:
# save random binary Linear Program with 10 vars

lp_bin = random_lp_int(15, 10, 0, 1)
model = lp_bin.createLP("test_bin")
with open(getPath(lp_bin.name), 'w+') as file:
    file.write(model.export_as_lp_string())

In [3]:
# read linear program from file
model = ModelReader.read(filename=getPath(lp_bin.name),model_name = lp_bin.name, model_class=AdvModel)
# solve with CPLEX
print(model.solve())

solution for: test_bin
objective: 1
x9=1



In [4]:
# check x which was used internally for random generation
all(lp_bin.x[i] == model.solution.get_all_values()[i] for i in range(0,len(lp_bin.x)))

False

In [5]:
lp_bin.x

array([1, 0, 1, 0, 1, 0, 0, 1, 1, 1])

In [6]:
# objective with lp_bin.x
lp_bin.c.dot(lp_bin.x)

2

In [7]:
# save random integer Linear Program with 5 vars

lp_int = random_lp_int(15, 5, -1, 3)
model = lp_int.createLP("test_int")
with open(getPath(lp_int.name), 'w+') as file:
    file.write(model.export_as_lp_string())

In [8]:
# read linear program from file
model = ModelReader.read(filename=getPath(lp_int.name),model_name = lp_int.name, model_class=AdvModel)
# solve with CPLEX
print(model.solve())

solution for: test_int
objective: -1
x0=1
x1=3
x2=-1
x4=1



# Solve with DWave Leap and CliqueSampler

In [9]:
from dwave.system import DWaveCliqueSampler, LeapHybridSampler

In [10]:
solvers = [createSolver(LeapHybridSampler()), createSolver(DWaveCliqueSampler(),1000)]
results = {}
qps = {lp_bin.name : lp_bin.model, lp_int.name : lp_int.model}
for qp_name in qps.keys():
    print(qp_name)

test_bin
test_int


In [11]:
qp_name = "test_bin"
results[qp_name] = [optimizer.solve(qps[qp_name]) for optimizer in solvers]
# problem ids : 
# Leap 977fc3a7-2f64-43a5-9576-a2cdeb91c2a2
# Advantage system a2a552cc-7fa1-45f1-956c-acbb67033c47

constraint A0leb0 is infeasible due to substitution
constraint A1leb1 is infeasible due to substitution
constraint A2leb2 is infeasible due to substitution
constraint A4leb4 is infeasible due to substitution
constraint A5leb5 is infeasible due to substitution
constraint A6leb6 is infeasible due to substitution
constraint A7leb7 is infeasible due to substitution
constraint A9leb9 is infeasible due to substitution
constraint A11leb11 is infeasible due to substitution
constraint A12leb12 is infeasible due to substitution
constraint A13leb13 is infeasible due to substitution
constraint A14leb14 is infeasible due to substitution


In [12]:
for res in results[qp_name]:
    print(res)

optimal function value: 1.0
optimal value: [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
status: SUCCESS
optimal function value: 2.0
optimal value: [1. 0. 1. 0. 1. 0. 0. 1. 1. 1.]
status: SUCCESS


## so we only found a solution with leap hybrid solver
## why does advantage test say success?
## lets try the integer problem

In [13]:
qp_name = "test_int"
results[qp_name] = [optimizer.solve(qps[qp_name]) for optimizer in solvers]
# problem ids : 
# Leap 0eae18c4-9db9-4706-9095-c838ffaea567
# Advantage system 47c7003b-44bb-41bd-b039-1c232b8c047e

constraint A0leb0 is infeasible due to substitution
constraint A1leb1 is infeasible due to substitution
constraint A2leb2 is infeasible due to substitution
constraint A3leb3 is infeasible due to substitution
constraint A4leb4 is infeasible due to substitution
constraint A5leb5 is infeasible due to substitution
constraint A6leb6 is infeasible due to substitution
constraint A7leb7 is infeasible due to substitution
constraint A8leb8 is infeasible due to substitution
constraint A9leb9 is infeasible due to substitution
constraint A10leb10 is infeasible due to substitution
constraint A11leb11 is infeasible due to substitution
constraint A12leb12 is infeasible due to substitution
constraint A13leb13 is infeasible due to substitution
constraint A14leb14 is infeasible due to substitution


In [14]:
for res in results[qp_name]:
    print(res)

optimal function value: -1.0
optimal value: [ 1.  3. -1.  0.  1.]
status: SUCCESS
optimal function value: -1.0
optimal value: [ 1.  3. -1. -1.  0.]
status: INFEASIBLE


## again, only leap hybrid was able to solve the problem